# Searching algorithms in Python by __[Artyom Iudin](https://github.com/Tomas542/DSaA)__

<div class="alert alert-block alert-info">
<b>Chapter navigation</b> is broken on github. Download .ipynb to use it.
</div>

[Link to my gitlab](https://gitlab.com/Tomas245/dsaa)

# Chapters
0. [Preparetions](#preps)
      1. [In bounds func](#in_bounds)
1. [Linear search](#linear)
2. [Binary search](#binary)
3. [Binary search tree](#binary_tree)
4. [Fibonacci search](#fibonacci)
5. [Interpolation search](#interpolation)
6. [Bloom filter](#bloom)
7. [Hash-based search](#hash_common)
      1. [With simple rehash](#hash_simple)
      2. [With random rehash](#hash_random)
      3. [With chain rule rehash](#hash_chain)

# Preparetions <a class="anchor" id="preps"></a>

### In bounds func <a class="anchor" id="in_bounds"></a>

When we have sorted structure we can check if target in bounds of it.

In [10]:
def in_bounds(array: list[int], target: int) -> bool:
    return array[0] <= target <= array[-1]

# Linear search <a class="anchor" id="linear"></a>

Check every element of array (or another structure). Only useful if array is not sorted.

Best case Scenario : O(1) (if it is first element)

Worst case Scenario : O(n) 

In [11]:
def linear_search(array: list[int], target: int) -> int:
    for i in range(len(array)):
        # we found it
        if array[i] == target:
            return i

# Binary search <a class="anchor" id="binary"></a>

Deviding array by two until finds the element

Worst case Scenario : O(log n) 

In [12]:
def binary_search(array: list[int], target: int) -> int:
    # initializing 2 variables for out boundaries
    low = 0
    high = len(array) - 1

    if not(in_bounds(array, target)):
        return -1

    # search until our boundaries won't be equal
    while low <= high:
        # calculating middle element that we will check
        mid = (low + high) // 2 # it can be '<< 1' instead of '// 2' 

        # if target lower move high bound 
        if array[mid] > target:
            high = mid - 1

        # if target higher move low target
        elif array[mid] < target:
            low = mid + 1

        # we found it
        elif array[mid] == target:
            return mid
    
    # didn't find
    return -1

# Binary search tree <a class="anchor" id="binary_tree"></a>

Building binary tree data structure to search in nodes.

Best case Scenario : O(log n) - balanced

Worst case Scenario : O(n) - unbalanced

In [13]:
class BSTNode:
    def __init__(self, value:int = None):
        # value stored in the node
        self.value = value
        # stored lower and higher values in nodes
        self.left = None
        self.right = None

    def __contains__(self, value:int) -> bool:
        # we found value
        if self.value == value:
            return True

        # we search on the left
        if self.value > value:
            if self.left == None:
                return False
            
            # go deeper
            return self.left.search(value)
        
        # we search on the right
        if self.right == None:
            return False
        
        # go deeper
        return self.right.search(value)

    def add(self, value:int) -> None:
        # if we got empty node
        if not self.value:
            self.value = value
            return 
        
        # already have this number
        if self.value == value:
            return 
        
        # move to the right node
        if self.value < value:
            # right node  isn't empty
            if self.right:
                self.right.add(value)
                return 
            
            # right node is empty
            self.right = BSTNode(value)
            return 
        
        # move to the left node
        if self.left:
            # left node isn't empty
            self.left.add(value)
            return
        
        # left node is empty
        self.left = BSTNode(value)

    def get_min(self) -> int:
        current = self
        
        # moing to the left nodes till the end
        while current.left is not None:
            current = current.left
        
        return current.value
    
    def get_max(self) -> int:
        current = self
        
        # moving to the right nodes till end
        while current.right is not None:
            current = current.right
        
        return current.value
    
    def delete(self, value:int):
        # empty tree
        if self == None:
            return self
        
        # searching on the right
        if self.value < value:
            # we can move to the right (it is not empty)
            if self.right:
                self.right = self.right.delete(value)

            # we didn't find the value in the tree
            return self
        
        # searching on the left
        if self.value > value:
            # we can move to the left (it is not empty)
            if self.left:
                self.left = self.left.delete(value)
            
            # we didn't find the value in the tree
            return  self
        
        # we found the value but we don't have value on the right
        if self.right == None:
            return self.left
        
        # we trying to replace deleting value but we don't have value on the left
        if self.left == None:
            return self.right
        
        # chosing smallest number after our deleting value. First go on the right
        min_larger_node = self.right

        # chosing smallest number after our deleting value. Second go to the left
        while min_larger_node.left:
            min_larger_node = min_larger_node.left
        
        # replace deleting value
        self.value = min_larger_node.value
        # remove replacing value
        self.right = self.right.delete(min_larger_node.value)
        return self
    
    # sorted values of the tree
    def inorder(self, values:list[int] = []) -> list[int]:
        if self.left is not None:
            self.left.inorder(values)

        if self.value is not None:
            values.append(self.value)

        if self.right is not None:
            self.right.inorder(values)
        
        return values

    # one of two orders to check structure of the tree
    def preorder(self, values:list[int] = []) -> list[int]:
        if self.value is not None:
            values.append(self.value)
        
        if self.left is not None:
            self.left.preorder(values)

        if self.right is not None:
            self.right.preorder(values)
        
        return values
    
    # one of two orders to check structure of the tree
    def postorder(self, values:list[int] = []) -> list[int]:
        if self.left is not None:
            self.left.postorder(values)

        if self.right is not None:
            self.right.postorder(values)
        
        if self.value is not None:
            values.append(self.value)

        return values

# Fibonacci search <a class="anchor" id="fibonacci"></a>

Like binary search, but we uses fibonacci numbers as indexes. Only useful if target in 1/3 of array. Subsearch could be any algorythm of search. I used binary search.

Average case Scenario: O(n/3) * we know target in 1/3 of array

Worst case Scenario: O(n)

In [14]:
def fibonacci_search(array:list[int], target:int) -> int:
    if not(in_bounds(array, target)):
        return -1
    
    # initialize 2 fibonacci numbers. fib1 contains low border, fib2 - high
    fib1 = 0
    fib2 = 1

    # searching for borders
    while array[fib2] <= target:
        # calulating new borders
        fib2, fib1 = fib1 + fib2, fib2

    # searching in the subarray with our borders. Could be another searching algorythms
    bs_ind = binary_search(array[fib1:fib2 + 1], target)
    
    # we didn't find our value
    if bs_ind == -1:
        return -1
    
    # case when our searched number is the last in the array
    return min(bs_ind + fib1, len(array))

# Interpolation search <a class="anchor" id="interpolation"></a>

Calculating approximate position of target. Looks similar to binary search.

Best and Average cases Scenario: O(log log n)

Worst case Scenario: O(n)

In [32]:
def interpolation_search(array:list[int], target: int) -> int:
    if not(in_bounds(array, target)):
        return -1
    
    # initialize 2 bounds
    low = 0
    high = len(array) - 1

    while low <= high and array[low] <= target:
        # calculating our middle with this formula
        middle = low + int(((high - low) / (array[high] - array[low])) * (target - array[low]))

        # going up cause target is higher
        if array[middle] < target:
            low = middle + 1
            
        # goind down cause target is lower
        elif array[middle] > target:
            high = middle - 1
        
        # found it
        else:
            return middle

    # didn't find
    return -1

# Bloom filter <a class="anchor" id="bloom"></a>

Analog of hash map. Can give false positive answer.

In [ ]:
class BloomFilter:
    def __init__(self, size:int = 1000, hash_func = None):
        # container of values
        self.bits = 0
        self.size = size

        if hash_func is None:
            # number of hash functions
            self.k = 1
            self.hash_func = [lambda e, size: hash(e) % size]
        
        else:
            # number of hash functions
            self.k = len(hash_func)
            self.hash_func = hash_func
    
    def add(self, value: int):
        for hf in self.hash_func:
            # bitwise manipulations
            self.bits |= 1 << hf (value, self.size)

    def __contains__(self, value:int) -> bool:
        for hf in self.hash_func:
            if self.bits & 1 << hf(value, self.size) == 0:
                return False

        return True

# Hash-based search <a class="anchor" id="hash_common"></a>

Search on hash sum. Includes 3 rehash methonds to avoid collision.

### With simple rehash <a class="anchor" id="hash_simple"></a>

Search on hash sum. Includes 3 rehash methonds to avoid collision.

### With random rehash <a class="anchor" id="hash_random"></a>

Search on hash sum. Includes 3 rehash methonds to avoid collision.

### With chain rule rehash <a class="anchor" id="hash_chain"></a>

Search on hash sum. Includes 3 rehash methonds to avoid collision.